In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_stock_prices.csv to cleaned_stock_prices.csv


In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
df = pd.read_csv("cleaned_stock_prices.csv")
def clean_volume(v):
    if isinstance(v, str):
        v = v.replace(",", "").strip()
        if v.endswith("M"):
            return float(v[:-1]) * 1_000_000
        if v.endswith("K"):
            return float(v[:-1]) * 1_000
        if v == "-" or v == "":
            return 0
    try:
        return float(v)
    except:
        return 0
if "Vol." in df.columns:
    df["Vol."] = df["Vol."].apply(clean_volume)
def clean_percent(p):
    if isinstance(p, str):
        p = p.replace("%", "")
        if p.strip() == "":
            return 0
        return float(p) / 100
    try:
        return float(p)
    except:
        return 0
if "Change %" in df.columns:
    df["Change %"] = df["Change %"].apply(clean_percent)
if "Date" in df.columns:
    df = df.drop(columns=["Date"])
df = df.select_dtypes(include=[np.number])
print("Columns after cleaning:", df.columns.tolist())
target = "Price" if "Price" in df.columns else df.columns[-1]
print("Using target:", target)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
WINDOW = 30
X, y = [], []
for i in range(len(scaled) - WINDOW):
    X.append(scaled[i:i+WINDOW])
    y.append(scaled[i+WINDOW][df.columns.get_loc(target)])
X = np.array(X)
y = np.array(y)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
lstm = Sequential([
    LSTM(64, input_shape=(WINDOW, X.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
print("\nTraining LSTM...")
lstm.fit(X_train, y_train, epochs=15, batch_size=32, verbose=1)
pred_lstm = lstm.predict(X_test)
mse_lstm = mean_squared_error(y_test, pred_lstm)
r2_lstm = r2_score(y_test, pred_lstm)
cnn = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(WINDOW, X.shape[2])),
    MaxPooling1D(2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1)
])
cnn.compile(optimizer='adam', loss='mse')
print("\nTraining 1D-CNN...")
cnn.fit(X_train, y_train, epochs=15, batch_size=32, verbose=1)
pred_cnn = cnn.predict(X_test)
mse_cnn = mean_squared_error(y_test, pred_cnn)
r2_cnn = r2_score(y_test, pred_cnn)
print("\n================= MODEL RESULTS =================")
print(f"LSTM → MSE: {mse_lstm:.6f},  R2: {r2_lstm:.6f}")
print(f"CNN  → MSE: {mse_cnn:.6f},  R2: {r2_cnn:.6f}")
print("\nDone!")


Columns after cleaning: ['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Using target: Price

Training LSTM...
Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.2960
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0713
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0344
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0130
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0151
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0115
Epoch 7/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0116
Epoch 8/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0106
Epoch 9/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0094
Epoch 10/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0099
Epoch 11/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0092
Epoch 12/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0083
Epoch 13/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0080
Epoch 14/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0081
Epoch 15/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0076


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step

Training 1D-CNN...
Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0585
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0214 
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0204 
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0156 
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0130 
Epoch 7/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0137
Epoch 8/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0118 
Epoch 9/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0116
Epoch 10/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0114 
Epoch 11/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0104
Epoch 12/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 
Epoch 13/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0083 
Epoch 14/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 
Epoch 15/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0073 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step

=======